In [ ]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2021-03-20 19:56:30--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  17.2MB/s    in 17s     

2021-03-20 19:56:48 (10.0 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2021-03-20 19:56:48--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  14.5MB/s    in 11s     

2021-03-20 19:56:59 (5.59 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [ ]:
import keras
from keras.layers.normalization import BatchNormalization
import numpy as np
import scipy.io as io
from keras.layers import Activation, Flatten,Softmax
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.models import Sequential
from keras import optimizers
from keras.backend import function
import pandas as pd

In [ ]:
from keras.backend import function

In [ ]:
#importing data
train_set = io.loadmat("train_32x32.mat")
test_set = io.loadmat("test_32x32.mat")
x_train = train_set['X']
y_train = train_set['y']
x_test = test_set['X']
y_test = test_set['y']
# Reshaping test and training data sets
x_train = np.moveaxis(x_train,[3,0,1,2],[0,1,2,3])
x_train = x_train.astype('float32')
x_test = np.moveaxis(x_test,[3,0,1,2],[0,1,2,3])
x_test = np.asarray(x_test)
x_test = x_test.astype('float32')
#Normalisation des données
x_train /= x_train.max()
x_test /= x_test.max()
# Passage de classe comme integer à classe comme vecteur de dimension classes
y_test_RN = [y-1 for y in y_test]# la classe 1 corresponds au array qui à 1 en indice 0
y_train_RN = [y-1 for y in y_train]
classes = 10
y_train_RN = keras.utils.to_categorical(y_train_RN, classes)
y_test_RN = keras.utils.to_categorical(y_test_RN, classes)
 #Définition du type de data pour optimiser la mémoire
y_train_RN = y_train_RN.astype('int32')
y_test_RN = y_test_RN.astype('int32')
# Featurewise normalization
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=False,
    zca_whitening=False, 
    zca_epsilon=1e-06)

datagen.fit(x_train)

In [ ]:
def deepnet2():
  model=Sequential()
  model.add(Conv2D(64, kernel_size=(3, 3),padding = 'same', activation='relu', input_shape=x_train.shape[1:]))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),  padding = 'valid', name='S1'))
  model.add(Conv2D(128, kernel_size=(3, 3), padding = 'valid', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),  padding = 'same', name='S2'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Dropout(0.5)) 
  model.add(Flatten())
  model.add(Dense(classes, activation='softmax'))
  adam = optimizers.Adam(lr = 0.001)
  model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer= adam,
                  metrics=['accuracy'])
  return model

In [ ]:

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
S1 (MaxPooling2D)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
S2 (MaxPooling2D)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 5, 5, 128)         147584    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 3, 128)         147584    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 1, 128)        

In [ ]:
model2.fit(x_train,y_train_RN, validation_split=0.2 ,epochs=10, batch_size=128)

Epoch 1/10
458/458 [==============================] - 256s 557ms/step - loss: 1.9635 - accuracy: 0.3014 - val_loss: 0.6204 - val_accuracy: 0.8182
Epoch 2/10
458/458 [==============================] - 256s 559ms/step - loss: 0.6761 - accuracy: 0.7991 - val_loss: 0.4275 - val_accuracy: 0.8718
Epoch 3/10
458/458 [==============================] - 259s 565ms/step - loss: 0.4720 - accuracy: 0.8639 - val_loss: 0.3664 - val_accuracy: 0.8924
Epoch 4/10
458/458 [==============================] - 256s 560ms/step - loss: 0.4030 - accuracy: 0.8847 - val_loss: 0.3381 - val_accuracy: 0.8993
Epoch 5/10
458/458 [==============================] - 253s 552ms/step - loss: 0.3588 - accuracy: 0.8989 - val_loss: 0.3142 - val_accuracy: 0.9072
Epoch 6/10
458/458 [==============================] - 253s 553ms/step - loss: 0.3082 - accuracy: 0.9124 - val_loss: 0.2925 - val_accuracy: 0.9186
Epoch 7/10
458/458 [==============================] - 253s 553ms/step - loss: 0.2716 - accuracy: 0.9220 - val_loss: 0.3049 -

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
model2.save('/content/drive/My Drive/Projet 3A')

INFO:tensorflow:Assets written to: /content/drive/My Drive/Projet 3A/assets


In [ ]:
model = keras.models.load_model('/content/drive/My Drive/Projet 3A')

In [ ]:
scores=[]
x_train_10000=x_train[:10000]

In [ ]:
y_test_KNN=[k.item() for k in y_test]
y_train_KNN=[k.item() for k in y_train]

# Couche conv2d 50

In [ ]:
model.layers[6]

In [ ]:
get_conv50_output = function([model.layers[0].input],
                                  [model.layers[6].output])
layer_output_conv50 = get_conv50_output([x_train_10000])

In [ ]:
np.array(layer_output_conv50[0]).shape

(10000, 1, 1, 128)

In [ ]:
conv50=np.array(layer_output_conv50[0]).reshape(10000,128)

In [ ]:
conv50.shape

(10000, 128)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_conv50=KNeighborsClassifier()
knn_conv50.fit(conv50,y_train_KNN[:10000])

NameError: ignored

In [ ]:
layer_output_conv50_test = get_conv50_output([x_test[:5000]])

In [ ]:
conv50_test=np.array(layer_output_conv50_test[0]).reshape(5000,128)

In [ ]:
score=knn_conv50.score(conv50_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)

In [ ]:
scores

[0.9142, 0.9142]

# Conv2D 40

In [ ]:
model.layers[5]

In [ ]:
%timeit
get_conv40_output = function([model.layers[0].input],
                                  [model.layers[5].output])
layer_output_conv40 = get_conv40_output([x_train_10000])

In [ ]:
np.array(layer_output_conv40[0]).shape

(10000, 3, 3, 128)

In [ ]:
conv40=np.array(layer_output_conv40[0]).reshape(10000,1152)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_conv40=KNeighborsClassifier()
knn_conv40.fit(conv40,y_train_KNN[:10000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
layer_output_conv40_test = get_conv40_output([x_test[:5000]])

In [ ]:
conv40_test=np.array(layer_output_conv40_test[0]).reshape(5000,1152)

In [ ]:
score=knn_conv40.score(conv40_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)

In [ ]:
scores

[0.9142, 0.9142, 0.881]

# Conv2D 30

In [ ]:
model.layers[4]

In [ ]:
get_conv30_output = function([model.layers[0].input],
                                  [model.layers[4].output])
layer_output_conv30 = get_conv30_output([x_train_10000])

In [ ]:
np.array(layer_output_conv30[0]).shape

(10000, 5, 5, 128)

In [ ]:
conv30=np.array(layer_output_conv30[0]).reshape(10000,3200)

In [ ]:
knn_conv30=KNeighborsClassifier()
knn_conv30.fit(conv30,y_train_KNN[:10000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
layer_output_conv30_test = get_conv30_output([x_test[:5000]])

In [ ]:
conv30_test=np.array(layer_output_conv30_test[0]).reshape(5000,3200)

In [ ]:
score=knn_conv30.score(conv30_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores

[0.9142, 0.9142, 0.881, 0.8164]

#Maxpooling S2

In [ ]:
model.layers[3]

In [ ]:
get_S2_output = function([model.layers[0].input],
                                  [model.layers[3].output])
layer_output_S2 = get_S2_output([x_train_10000])

In [ ]:
np.array(layer_output_S2[0]).shape

(10000, 7, 7, 128)

In [ ]:
S2=np.array(layer_output_S2[0]).reshape(10000,6272)

In [ ]:
knn_S2=KNeighborsClassifier()
knn_S2.fit(S2,y_train_KNN[:10000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
layer_output_S2_test = get_S2_output([x_test[:5000]])

In [ ]:
S2_test=np.array(layer_output_S2_test[0]).reshape(5000,6272)

In [ ]:
score=knn_S2.score(S2_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores

[0.9142, 0.9142, 0.881, 0.8164, 0.6956]

# Conv2D 20

In [ ]:
model.layers[2]

In [ ]:
get_conv20_output = function([model.layers[0].input],
                                  [model.layers[2].output])
layer_output_conv20 = get_conv20_output([x_train_10000])

In [ ]:
np.array(layer_output_conv20[0]).shape

(10000, 14, 14, 128)

In [ ]:
conv20=np.array(layer_output_conv20[0]).reshape(10000,25088)

In [ ]:
knn_conv20=KNeighborsClassifier()
knn_conv20.fit(conv20,y_train_KNN[:10000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
layer_output_conv20_test = get_conv20_output([x_test[:5000]])

In [ ]:
conv20_test=np.array(layer_output_conv20_test[0]).reshape(5000,25088)

In [ ]:
score=knn_conv20.score(conv20_test,y_test_KNN[:5000])

In [ ]:
scores.append(score)
scores

[0.9142, 0.9142, 0.881, 0.8164, 0.6956, 0.653]

# Maxpooling S1

In [ ]:
model.layers[1]

In [ ]:
get_S1_output = function([model.layers[0].input],
                                  [model.layers[1].output])
layer_output_S1 = get_S2_output([x_train_10000])

les scores [0.9142, 0.9142, 0.881, 0.8164, 0.6956, 0.653]